# Vehicle and pedestrian stops in Vermont

Our data consists of vehicle and pedestrian stops from law enforcement departments in Vermont. Using this dataset we will try to predict the outcome (whether it results in an arrest) of the traffic stop using its features.

In [1]:
# Standard Headers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
# Enable inline mode for matplotlib so that Jupyter displays graphs
%matplotlib inline


In [2]:
#read the data
data = pd.read_csv('vermont.csv', header='infer')
print('Number of rows vs Number of columns')
data.shape

Number of rows vs Number of columns


(283285, 23)

### Data Cleaning


The dataset is missing values in many columns. We can see how many values in each column are missing.

In [3]:
#All missing values for each column
pd.isnull(data).sum()

id                            0
state                         0
stop_date                     0
stop_time                     0
location_raw                694
county_name                 705
county_fips                 705
fine_grained_location       347
police_department             0
driver_gender              1712
driver_age_raw             1171
driver_age                 1286
driver_race_raw            3984
driver_race                4817
violation_raw              2178
violation                  2178
search_conducted              0
search_type_raw            2240
search_type              279866
contraband_found             34
stop_outcome               2325
is_arrested                   0
officer_id                   12
dtype: int64

The size of the dataset is big and some features are needed to build an accurate model, therefore we will delete all records with missing categorical data.
For some numerical data, we will do imputation, using the mean of the column.

In [4]:
# Location is needed to build an accurate model, so all missing data will be dropped
data = data.dropna(subset = ['county_name'])

# Driver's gender is also needed, so will drop records without gender
data = data.dropna(subset = ['driver_gender'])

# Driver's race is also needed, so will drop records without race
data = data.dropna(subset = ['driver_race'])

# Violation is also needed, so will drop records without violation
data = data.dropna(subset = ['violation'])

# Search Type Raw, Contraband Found, Stop Outcome and Officer ID only include 500 records for missing values, so we will drop these fields too
data = data.dropna(subset = ['search_type_raw'])
data = data.dropna(subset = ['contraband_found'])
data = data.dropna(subset = ['stop_outcome'])
data = data.dropna(subset = ['officer_id'])

# Driver's age is important but to avoid dropping further data we will fill this in with average age
data['driver_age']=data['driver_age'].astype(float) #convert data from strings to floats
mean = data['driver_age'].mean()
print("mean=", mean)
data['driver_age'].fillna(mean, inplace=True)

pd.isnull(data).sum()

mean= 38.805525581327295


id                            0
state                         0
stop_date                     0
stop_time                     0
location_raw                  0
county_name                   0
county_fips                   0
fine_grained_location       202
police_department             0
driver_gender                 0
driver_age_raw              767
driver_age                    0
driver_race_raw               0
driver_race                   0
violation_raw                 0
violation                     0
search_conducted              0
search_type_raw               0
search_type              270902
contraband_found              0
stop_outcome                  0
is_arrested                   0
officer_id                    0
dtype: int64

In [5]:
data.shape


(274201, 23)

Columns 'is_arrested' and 'stop_outcome' contain the same data in a different form. One of those columns needs to be deleted, otherwise, we would have data leakage. Choosing 'stop_outcome' would lead to a multiclass classification; therefore we will do classification on column 'is_arrested'.

In [6]:
pd.crosstab(data.stop_outcome, data.is_arrested, normalize='index')
#columns is_arrested and stop_outcome have the same data, one of these needs to be deleted

is_arrested,False,True
stop_outcome,,
Arrest for Violation,0.0,1.0
Citation,1.0,0.0
Verbal Warning,1.0,0.0
Warrant Arrest,0.0,1.0
Written Warning,1.0,0.0


From the correlation table, we see that the column that has the greatest correlation with the class label is column 'search_conducted'.

In [7]:
data.corr()

,county_fips,driver_age_raw,driver_age,search_conducted,is_arrested,officer_id
county_fips,1.000000,0.023329,0.023391,0.000467,-0.000168,-0.020262
driver_age_raw,0.023329,1.000000,0.999646,-0.075187,-0.043471,-0.006270
driver_age,0.023391,0.999646,1.000000,-0.074852,-0.043433,-0.006253
search_conducted,0.000467,-0.075187,-0.074852,1.000000,0.269260,0.010091
is_arrested,-0.000168,-0.043471,-0.043433,0.269260,1.000000,0.013594
officer_id,-0.020262,-0.006270,-0.006253,0.010091,0.013594,1.000000


We can take a closer look at the relationship between these two columns ('is_arrested' and 'search_conducted'). From the cross table we see that only 0.08% drivers for whom search was not conducted were arrested. On the other side, 27% of the drivers that were arrested were searched as well.

In [8]:
pd.crosstab(data.search_conducted, data.is_arrested, normalize='index')

is_arrested,False,True
search_conducted,,
False,0.991237,0.008763
True,0.722340,0.277660


Using the same table, we can see the relationship between the driver's race and whether they were arrested or not. It shows that a greater percentage of Black and Hispanic drivers were arrested compared to White and Asian drivers.

In [9]:
pd.crosstab(data.driver_race, data.is_arrested, normalize = 'index')

is_arrested,False,True
driver_race,,
Asian,0.993638,0.006362
Black,0.978354,0.021646
Hispanic,0.982463,0.017537
Other,0.973585,0.026415
White,0.988202,0.011798


The feature 'id' would not make sense to use as a predictive feature. As the whole dataset is made in Vermont, the column 'state' is the same in all rows. Therefore we can drop these two columns. Some columns are doubled, meaning that there is raw data that the officer wrote, and the data with values that are unified in the state. That means that the actual data (the one that is not raw) is more consistent, but the raw data has more variability. Looking at the values of these columns, we can see if raw data is more useful than the actual data, and choose which one to use for a certain feature.


In [10]:
features_X = data.drop(['id','state','fine_grained_location','stop_date','driver_age_raw','stop_outcome','violation_raw','search_type','county_name','driver_race','officer_id', 'is_arrested'], axis = 1)
features_X.corr()

classLabel_Y = data['is_arrested']
features_X.head()

,stop_time,location_raw,county_fips,police_department,driver_gender,driver_age,driver_race_raw,violation,search_conducted,search_type_raw,contraband_found
0,00:10,East Montpelier,50023.0,MIDDLESEX VSP,M,22.0,White,Moving violation,False,No Search Conducted,False
3,00:11,Whiting,50001.0,NEW HAVEN VSP,F,18.0,White,Moving violation,False,No Search Conducted,False
4,00:35,Hardwick,50005.0,ROYALTON VSP,M,18.0,White,Moving violation,False,No Search Conducted,False
5,00:44,Hardwick,50005.0,ROYALTON VSP,F,20.0,White,Equipment,False,No Search Conducted,False
8,01:10,Rochester,50027.0,ROCKINGHAM VSP,M,24.0,Black,Moving violation,False,No Search Conducted,False


There are features that are categorical. We need to convert them to numbers to be able to make models on the dataset.

In [11]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Converting all labels to numbers
features_X['location_raw']= label_encoder.fit_transform(features_X['location_raw']) 
features_X['county_fips']= label_encoder.fit_transform(features_X['county_fips'])
features_X['police_department']= label_encoder.fit_transform(features_X['police_department'])
features_X['driver_gender']= label_encoder.fit_transform(features_X['driver_gender'])
features_X['driver_race_raw']= label_encoder.fit_transform(features_X['driver_race_raw'])
features_X['violation']= label_encoder.fit_transform(features_X['violation'])
features_X['search_type_raw']= label_encoder.fit_transform(features_X['search_type_raw'])
features_X['contraband_found']= label_encoder.fit_transform(features_X['contraband_found'])
#features_X['stop_outcome']= label_encoder.fit_transform(features_X['stop_outcome'])
features_X['stop_time']= label_encoder.fit_transform(features_X['stop_time'])

#features_X['is_arrested']= label_encoder.fit_transform(features_X['is_arrested'])

features_X.head()

,stop_time,location_raw,county_fips,police_department,driver_gender,driver_age,driver_race_raw,violation,search_conducted,search_type_raw,contraband_found
0,10,88,11,3,1,22.0,4,2,False,3,0
3,11,324,0,4,0,18.0,4,2,False,3,0
4,35,128,2,6,1,18.0,4,2,False,3,0
5,44,128,2,6,0,20.0,4,1,False,3,0
8,70,234,13,5,1,24.0,1,2,False,3,0


Since the dataset is imbalanced, we need to balance it. We will oversample the minority class with using Smote, and then undersample the majority class.

In [12]:
import imblearn
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
#number of true/false in class label before balancing data
counter = Counter(classLabel_Y)
print(counter)
#the data needs to be balanced
#oversampeling the minority class with smote

oversample = SMOTE(sampling_strategy=0.015)
data_X, data_Y = oversample.fit_resample(features_X, classLabel_Y)
#undersampeling the majority class
under = RandomUnderSampler(sampling_strategy=0.9)
data_X, data_Y = under.fit_resample(data_X, data_Y)
#number of true/false in class label afrer balancing data
counter = Counter(data_Y)
print(counter)

data_X = pd.DataFrame(data_X)
data_Y = pd.DataFrame(data_Y)


data_X.shape


Counter({False: 270911, True: 3290})
Counter({False: 4514, True: 4063})


(8577, 11)

### Data modeling

#### Naive Bayes
We will use Naive Bayes to make a model, because the features are independent from each other.

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
#classifing with naive bayes
clf = GaussianNB()
scores = cross_val_score(clf, data_X, data_Y, cv=10) 
predicts = cross_val_predict(clf, data_X, data_Y, cv=10) 

print("Accuracy:", scores.mean()*100)
print(classification_report(data_Y, predicts))


Accuracy: 66.6195844808326
              precision    recall  f1-score   support

       False       0.61      0.99      0.76      4514
        True       0.96      0.31      0.47      4063

    accuracy                           0.67      8577
   macro avg       0.79      0.65      0.61      8577
weighted avg       0.78      0.67      0.62      8577



#### K Nearest Neighbors
For k nearest neighbors classifier we need to use scaler and PCA dimensionality reduction.

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


scaler = sk.preprocessing.MinMaxScaler()
pca = PCA()
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('knn', knn)])

param_grid = {
    'pca__n_components': list(range(1, 11)),
    'knn__n_neighbors': list(range(1, 10)),  
}

grid_search = GridSearchCV( pipe, param_grid, cv=5, scoring='accuracy')
grid_search.fit(data_X, data_Y)

print(grid_search.best_params_)

nested_score = cross_val_score(grid_search, data_X, data_Y, cv=5)
print("Accuracy:", nested_score.mean()*100)


predicts = cross_val_predict(grid_search, data_X, data_Y, cv=5) 
print(classification_report(data_Y, predicts))


{'knn__n_neighbors': 8, 'pca__n_components': 9}
Accuracy: 72.81120568601816
              precision    recall  f1-score   support

       False       0.71      0.82      0.76      4514
        True       0.76      0.63      0.69      4063

    accuracy                           0.73      8577
   macro avg       0.73      0.72      0.72      8577
weighted avg       0.73      0.73      0.73      8577



#### Neural Networks

In [15]:
from sklearn.neural_network import MLPClassifier


scaler = sk.preprocessing.MinMaxScaler()
clf = MLPClassifier()

pipe =  Pipeline(steps=[('scaler', scaler), ('MLP', clf)])

param_grid = {'MLP__hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,), (60,),(70,)],
             'MLP__activation' : ['identity', 'logistic', 'tanh', 'relu']}

grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid_search.fit(data_X, data_Y)
print(grid_search.best_params_)
nested_score = cross_val_score(grid_search, data_X, data_Y, cv=5)

print("Accuracy:", nested_score.mean()*100)

predicts = cross_val_predict(grid_search, data_X, data_Y, cv=5) 
print(classification_report(data_Y, predicts))

{'MLP__activation': 'relu', 'MLP__hidden_layer_sizes': (60,)}
Accuracy: 74.33865139456547
              precision    recall  f1-score   support

       False       0.72      0.84      0.78      4514
        True       0.78      0.64      0.71      4063

    accuracy                           0.75      8577
   macro avg       0.75      0.74      0.74      8577
weighted avg       0.75      0.75      0.74      8577



#### Support Vector Machines 
For support vector machines we need to use scaler and PCA dimensionality reduction.

In [16]:
from sklearn.svm import SVC 

scaler = sk.preprocessing.MinMaxScaler()
clf = SVC()
pca = PCA()
pipe =  Pipeline(steps=[('scaler', scaler), ('pca',pca), ('SVM', clf)])

param_grid = {'SVM__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
             'pca__n_components': list(range(1, 11))}

grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid_search.fit(data_X, data_Y)
print(grid_search.best_params_)

nested_score = cross_val_score(grid_search, data_X, data_Y, cv=5)
print("Accuracy:", nested_score.mean()*100)
predicts = cross_val_predict(grid_search, data_X, data_Y, cv=5) 
print(classification_report(data_Y, predicts))

{'SVM__kernel': 'rbf', 'pca__n_components': 7}
Accuracy: 69.21996640592596
              precision    recall  f1-score   support

       False       0.64      0.97      0.77      4514
        True       0.91      0.39      0.54      4063

    accuracy                           0.69      8577
   macro avg       0.78      0.68      0.66      8577
weighted avg       0.77      0.69      0.66      8577



#### Ensemble Classifiers

In [17]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier()

par = {"n_estimators": range(10,200,20) }

grid_search = GridSearchCV(clf, par, cv=5, scoring='accuracy')
grid_search.fit(data_X, data_Y)

print(grid_search.best_params_)
nested_score = cross_val_score(grid_search, data_X, data_Y, cv=5)

print("Accuracy:", nested_score.mean()*100)
predicts = cross_val_predict(grid_search, data_X, data_Y, cv=5) 
print(classification_report(data_Y, predicts))

{'n_estimators': 110}
Accuracy: 74.08177863460887
              precision    recall  f1-score   support

       False       0.71      0.86      0.78      4514
        True       0.80      0.61      0.69      4063

    accuracy                           0.74      8577
   macro avg       0.75      0.73      0.73      8577
weighted avg       0.75      0.74      0.74      8577



In [18]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()


par = {"max_depth": range(30,45) , 
      "min_samples_leaf": [9,12,15,18],
       "max_features" : ['sqrt', 'log2']}

grid_search = GridSearchCV(clf, par, cv=5, scoring='accuracy')
grid_search.fit(data_X, data_Y)

print(grid_search.best_params_)
nested_score = cross_val_score(grid_search, data_X, data_Y, cv=5)

print("Accuracy:", nested_score.mean()*100)
predicts = cross_val_predict(grid_search, data_X, data_Y, cv=5) 
print(classification_report(data_Y, predicts))

{'max_depth': 44, 'max_features': 'sqrt', 'min_samples_leaf': 9}
Accuracy: 75.03829193770375
              precision    recall  f1-score   support

       False       0.72      0.86      0.78      4514
        True       0.80      0.62      0.70      4063

    accuracy                           0.75      8577
   macro avg       0.76      0.74      0.74      8577
weighted avg       0.76      0.75      0.74      8577

